In [1]:
import os
os.chdir('../')
os.getcwd()

'e:\\github_clone\\Deep-Inverse-Reinforcement-Learning'

In [2]:
import os
import torch
from datasets import load_dataset
from algorithms import BehaviorClonning

In [3]:
dataset = load_dataset('NathanGavenski/LunarLander-v2')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['obs', 'actions', 'rewards', 'episode_starts'],
        num_rows: 383994
    })
})


In [4]:
full_dataset = dataset['train'].train_test_split(test_size=0.02, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['obs', 'actions', 'rewards', 'episode_starts'],
    num_rows: 376314
})
Dataset({
    features: ['obs', 'actions', 'rewards', 'episode_starts'],
    num_rows: 7680
})


# Data preparation

In [5]:
def prepare_data(dataset):
    states = []
    actions = []
    for data in dataset:
        states.append(data['obs'])  # Assuming 'state' contains the input features
        actions.append(data['actions'])  # Assuming 'action' is the label

    # Convert lists to PyTorch tensors
    X = torch.tensor(states, dtype=torch.float32)
    y = torch.tensor(actions, dtype=torch.long)  # Ensure correct data type for classification

    return X, y

In [6]:
X_train, y_train = prepare_data(dataset_train)
X_valid, y_valid = prepare_data(dataset_valid)

In [7]:
input_dim = X_train.shape[1]  # Number of features in state
output_dim = len(set(y_train.numpy()))  
print(f"input dim : {input_dim} \naction dim : {output_dim}")

input dim : 8 
action dim : 4


# Model initialization

In [15]:
bc_model = BehaviorClonning(input_dim, output_dim, learning_rate=1e-5)

# Training

In [17]:
bc_model.train(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50, Loss: 0.27027422189712524, Training Accuracy: 0.9489
Model saved at artifacts\model.pth
Epoch 2/50, Loss: 0.14782539010047913, Training Accuracy: 0.9499
Model saved at artifacts\model.pth
Epoch 3/50, Loss: 0.1551453173160553, Training Accuracy: 0.9504
Model saved at artifacts\model.pth
Epoch 4/50, Loss: 0.1254883110523224, Training Accuracy: 0.9515
Model saved at artifacts\model.pth
Epoch 5/50, Loss: 0.19697046279907227, Training Accuracy: 0.9541
Model saved at artifacts\model.pth
Epoch 6/50, Loss: 0.15270306169986725, Training Accuracy: 0.9541
Model saved at artifacts\model.pth
Epoch 7/50, Loss: 0.2548691928386688, Training Accuracy: 0.9574
Model saved at artifacts\model.pth
Epoch 8/50, Loss: 0.30900970101356506, Training Accuracy: 0.9575
Model saved at artifacts\model.pth
Epoch 9/50, Loss: 0.14011250436306, Training Accuracy: 0.9585
Model saved at artifacts\model.pth
Epoch 10/50, Loss: 0.17924895882606506, Training Accuracy: 0.9600
Model saved at artifacts\model.pth
Epoch

In [18]:
bc_model.evaluate(X_test=X_valid, y_test=y_valid)

0.9811197916666666

In [34]:
bc_model.predict(X_valid[0].unsqueeze(0)).item()

2

In [37]:
import gym
env = gym.make('LunarLander-v2')
obs, _ = env.reset()

In [42]:
torch.Tensor(obs).unsqueeze(0).shape

torch.Size([1, 8])